In [1]:
import timm
from torch import nn
from timm.data.loader import create_loader
import torch.utils.data as data
import torch
import pandas as pd

from sklearn.metrics import balanced_accuracy_score


from imp import reload
from collections import defaultdict
from types import SimpleNamespace


import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter
from timm.optim import create_optimizer
import pickle

In [2]:
from timm.data import (
    resolve_data_config,
)
from timm.models import create_model

In [23]:
!pwd

/aditya/chipbrain/cleanlab/celaba


In [4]:
df = pd.read_csv("img_align_celeba/all.csv")

set_lab = {}
for i,row in df.iterrows():
    q = str(row.tolist()[1:])
    if q not in set_lab:
        set_lab[(str(q))]=len(set_lab)

def get_lab(row):
    q = str(row.tolist()[1:])
    return set_lab[q]

df['unique_label'] = df.apply(get_lab,axis=1)


cnt = Counter(df['unique_label'])

def drop(val):
    if cnt[val]>10:
        return True
    return False

is_drop = df['unique_label'].apply(lambda x:drop(x))



df[is_drop].to_csv("img_align_celeba/all2.csv",index=False)

df = pd.read_csv("img_align_celeba/all2.csv")

In [5]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import accuracy_score

In [6]:

class MultiLabelModel(nn.Module):
    def __init__(self, model, n_classes,class_weights = None):
        super().__init__()
        self.base_model = model
        self.num_classes = n_classes

    def forward(self, x):
        x = self.base_model(x)
        
        x = torch.flatten(x, 1)

        return x

    def get_loss(self,loss_fn, output, target):
        
        return loss_fn(output,target)

    def validate(self,loader):
        self.eval();
        with torch.no_grad():
            total_loss = 0
            m = nn.Sigmoid()
            ops = []
            labels = []
            preds = []
            for batch_idx, (input, target) in enumerate(loader):
                input = input.cuda()
                labels.append(target.detach().cpu())
                target = target.float().cuda()
                output = m(self(input))
                loss = self.get_loss(loss_fn, output, target)

                total_loss += loss.item()
                pred_model = (output>0.5).detach().cpu()
                preds.append(pred_model)

            num_of_batches_per_epoch = len(loader)
            avg_loss = total_loss / num_of_batches_per_epoch
            print("VALIDATION DATA STATS")

            print("AVERAGE LOSS:",avg_loss)
            preds = torch.cat(preds).int()
            labels = torch.cat(labels).int()
            acc_score = accuracy_score(labels,preds)
            print("MULTILABEL accuracy score:",acc_score)
            per_class = []
            for i in range(len(preds.T)):
                per_class.append(accuracy_score(labels.T[i],preds.T[i]))
            print(dataset_train.label_names)
            print(per_class)
            print('\n\n')
        return avg_loss
    
    
    
    
    
    def predict_proba(self,loader):
        self.eval();
        with torch.no_grad():
            total_loss = 0
            m = nn.Sigmoid()
            ops = []
            labels = []
            preds = []
            for batch_idx, (input, target) in enumerate(loader):
                input = input.cuda()
                output = m(self(input))
                pred_model = output.detach().cpu()
                preds.append(pred_model)

            num_of_batches_per_epoch = len(loader)
            avg_loss = total_loss / num_of_batches_per_epoch
            preds = torch.cat(preds)
        return preds
    
    def fit(self,loader_train,load_val, num_epochs = 10 ):
        if os.path.exists("weights_model"):
            print("removing weights directory")
            os.system('rm -rf weights_model')
        os.mkdir("weights_model")
        args = SimpleNamespace()
        args.weight_decay = 0
        args.lr = 1e-4
        args.opt = 'adam' 
        args.momentum = 0.9
        args.sched = "step"
            
        optimizer = create_optimizer(args, self)
        saver = CheckpointSaver(
            model=self,
            optimizer=optimizer,
            checkpoint_dir="weights_model"
            )
        errs = []
        num_of_data_train = len(loader_train.dataset.data)
        for epoch in range(0,num_epochs):
            loss_train = train_one_epoch(
            num_of_data_train,
            epoch,
            self,
            loader_train,
            optimizer,
            loss_fn,
        )
            loss_val = self.validate(loader_val)
            errs.append([loss_train,loss_val])
            saver.save_checkpoint(epoch,metric=loss_val)
    
from PIL import Image


In [7]:
def train_one_epoch(
        num_of_data_train,
        epoch,
        model,
        loader,
        optimizer,
        loss_fn,
        lr_scheduler=None,
        saver=None,
        output_dir="",
        ):
        sta = time.time()
        second_order = hasattr(optimizer, "is_second_order") and optimizer.is_second_order
        model.train()
        total_loss = 0
        m = nn.Sigmoid()
        ops = []
        labels = []
        preds = []
        ct=0
        for batch_idx, (input, target) in enumerate(loader):
                input = input.cuda()
                ct+=1
                labels.append(target.detach().cpu())
                target = target.float().cuda()
                output = m(model(input))
                loss = model.get_loss(loss_fn, output, target)
                total_loss += loss.item()
                pred_model = (output>0.5).detach().cpu()
                preds.append(pred_model)
                optimizer.zero_grad()
                loss.backward(create_graph=second_order)
                optimizer.step()
                if ct%80==0:
                    print("LOSS:",loss.item())
        num_of_batches_per_epoch = len(loader)
        avg_loss = total_loss / num_of_batches_per_epoch
        print("TRAINING DATA STATS")
        print("AVERAGE LOSS:",avg_loss)
        preds = torch.cat(preds).int()
        labels = torch.cat(labels).int()
        acc_score = accuracy_score(labels,preds)
        print("MULTILABEL accuracy score:",acc_score)
        per_class = []
        for i in range(len(preds.T)):
            per_class.append(accuracy_score(labels.T[i],preds.T[i]))
        print(dataset_train.label_names)
        print(per_class)
        print('\n\n')
        sto = time.time()
        print("training time",sto-sta)
        return avg_loss

In [8]:
loss_fn = nn.BCELoss()


In [9]:
class DatasetMultiLabel(data.Dataset):
    def __init__(
            self,
            annotation_path=None,
            df = None,
            transform=None):

        super().__init__()
        self.transform = transform
        self.data = []
        self.labels = []
        self.label_names = []
        if annotation_path is None:
            assert df is not None
        else:
            df = pd.read_csv(annotation_path)
        
        cols = df.columns
        self.label_names = list(cols[1:-1])
        for i,row in df.iterrows():
            lb = []
            for j in cols:
                if j=='unique_label':
                    continue
                if j=='image_path':
                    self.data.append(row[j])
                else:
                    lb.append(float(row[j]))
            self.labels.append(lb)
                
    def __getitem__(self, idx):
        img_path = self.data[idx]

        img = Image.open(img_path)
        if self.transform:
            img = self.transform(img)

        labels = self.labels[idx]

        return img, labels

    def __len__(self):
        return len(self.data)

In [10]:
import os
from collections import Counter
import time

In [11]:
dataset = DatasetMultiLabel(df = df)

In [12]:
model = create_model(
    'efficientnet_b0',
    num_classes=len(dataset.labels[0]),
)
data_config = resolve_data_config(
       args = {}, model=model
    )

model = MultiLabelModel(
        model,
        n_classes=len(dataset.labels[0]),
    ).cuda()

In [13]:
from sklearn.model_selection import StratifiedKFold

In [25]:
num_splits = 4

In [14]:
skf = StratifiedKFold(n_splits=num_splits)

In [15]:
def reset_weights(m):
  '''
    Try resetting model weights to avoid
    weight leakage.
  '''
  for layer in m.children():
    if hasattr(layer, 'reset_parameters'):
#         print(f'Reset trainable parameters of layer = {layer}')
        layer.reset_parameters()

In [16]:
def create_df(pred_probs,dataset):
    ls = dataset_val.label_names
    cl = defaultdict(list)
    cl['image_loc'] = dataset.data
    for i in range(0,len(ls)):
        cl[ls[i]] = pred_val.T[i].tolist()
    return pd.DataFrame.from_dict(cl)

In [17]:
# pred_val = model.predict_proba(loader_val)

In [18]:
from timm.utils import CheckpointSaver


In [ ]:
ct = 1
for train_index, test_index in skf.split(df,df['unique_label']):
    if ct!=1:
        model.apply(reset_weights);
    dataset_train = DatasetMultiLabel(df = df.loc[train_index])
    dataset_val = DatasetMultiLabel(df = df.loc[test_index])
    loader_train = create_loader(
        dataset_train,
        input_size=data_config["input_size"],
        batch_size=64,
        is_training=True,
        mean=data_config["mean"],
        std=data_config["std"],
       interpolation=data_config["interpolation"],
    )
    loader_val = create_loader(
        dataset_val,
        input_size=data_config["input_size"],
        batch_size=64,
        is_training=False,
        mean=data_config["mean"],
        std=data_config["std"],
        interpolation=data_config["interpolation"],

    )
    model.fit(loader_train,loader_val,num_epochs=40)
    checkpoint = torch.load("weights_model/model_best.pth.tar")
    model.load_state_dict(checkpoint['state_dict'])
    pred_val = model.predict_proba(loader_val)
    df_pred = create_df(pred_val,dataset_val)
    df_pred.to_csv(str(ct)+"_fold.csv",index=False)
    ct+=1
    


LOSS: 0.43037253618240356
LOSS: 0.39355307817459106
LOSS: 0.395964115858078
LOSS: 0.3461613357067108
LOSS: 0.36866986751556396
LOSS: 0.4578284025192261
LOSS: 0.42840054631233215
LOSS: 0.3487946689128876
LOSS: 0.4757138788700104
LOSS: 0.3831671476364136
LOSS: 0.38085877895355225
LOSS: 0.3592175245285034
LOSS: 0.34022796154022217
LOSS: 0.3761342465877533
LOSS: 0.32846173644065857
LOSS: 0.42721328139305115
LOSS: 0.4486467242240906
LOSS: 0.3618222773075104
LOSS: 0.38367760181427
LOSS: 0.35117629170417786
LOSS: 0.3657556474208832
LOSS: 0.3876534402370453
LOSS: 0.38139352202415466
LOSS: 0.3599325716495514
LOSS: 0.3820578157901764
LOSS: 0.3913615942001343
LOSS: 0.3237290382385254
TRAINING DATA STATS
AVERAGE LOSS: 0.390529101273739
MULTILABEL accuracy score: 0.2653558476881233
['Eyeglasses', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Necklace', 'Wearing_Necktie', 'No_Beard', 'Smiling']
[0.9210250453309157, 0.7900257819582955, 0.9426988893925657, 0.8647084655485041, 0.9203521645512239, 0.89570

In [20]:
os

<module 'os' from '/usr/lib/python3.8/os.py'>

In [30]:
dfl = []
for i in range(1,num_splits+1):
    dfl.append(pd.read_csv(str(i)+"_fold.csv"))
    

cols = dfl[0].columns[1:]

df_pred = pd.concat(dfl,axis=0)

In [31]:
pred_probs = df_pred[cols].to_numpy()

In [32]:
np.save(open('pred_probs.npy','wb'),pred_probs)

In [36]:
data = list(df_pred['image_loc'])

In [38]:
pickle.dump(data,open("data.p",'wb'))

<module 'pickle' from '/usr/lib/python3.8/pickle.py'>